In [2]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 23.2 MB/s 
     |████████████████████████████████| 101 kB 14.5 MB/s 
     |████████████████████████████████| 6.6 MB 61.6 MB/s 
     |████████████████████████████████| 596 kB 62.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 28.8 MB/s 
     |████████████████████████████████| 140 kB 71.4 MB/s 
     |████████████████████████████████| 212 kB 72.8 MB/s 
     |████████████████████████████████| 1.1 MB 58.9 MB/s 
     |████████████████████████████████| 127 kB 71.3 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 74.5 MB/s 
     |███████████

In [3]:
from datasets import load_dataset

# 加载 ELI5 数据集的前 5000 行
eli5 = load_dataset("eli5", split="train_asks[:5000]")

Downloading:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

Dataset eli5 downloaded and prepared to /root/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa. Subsequent calls will reuse this data.


In [4]:
# 将数据集拆分为训练集和测试集
eli5 = eli5.train_test_split(test_size=0.2)

In [5]:
# 查看其中一个数据
eli5["train"][0]

{'answers': {'a_id': ['cvyz4ty'],
  'score': [3],
  'text': ['Very similar to the colors depicted in [pictures of a cross section of the Earth](_URL_2_), because the colors will be that of a [Black Body](_URL_0_). Each layer will have the same color as this [chart](_URL_3_); the core for example is ~7400 Kelvin which "white hot" while parts of the mantle are a dull red.\n\nPretty much the same colors you see [here](_URL_1_).']},
 'answers_urls': {'url': ['https://en.wikipedia.org/wiki/Black-body_radiation',
   'https://www.youtube.com/watch?v=VydPQuLyEns',
   'https://www.google.com/search?q=cross-section+of+earth',
   'https://upload.wikimedia.org/wikipedia/commons/0/00/Blackbody-colours-vertical.svg']},
 'document': '',
 'q_id': '3onkte',
 'selftext': '[deleted]',
 'selftext_urls': {'url': []},
 'subreddit': 'askscience',
 'title': 'If Earth was cut in half, what colours would the cross section be?',
 'title_urls': {'url': []}}

In [6]:
from transformers import AutoTokenizer

# 加载 DistilGPT2 分词器
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", use_fast=True)

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [7]:
# 使用 `flatten` 方法将 text 字段从嵌套结构中提取出来：
eli5 = eli5.flatten()
eli5["train"][0]

{'answers.a_id': ['cvyz4ty'],
 'answers.score': [3],
 'answers.text': ['Very similar to the colors depicted in [pictures of a cross section of the Earth](_URL_2_), because the colors will be that of a [Black Body](_URL_0_). Each layer will have the same color as this [chart](_URL_3_); the core for example is ~7400 Kelvin which "white hot" while parts of the mantle are a dull red.\n\nPretty much the same colors you see [here](_URL_1_).'],
 'answers_urls.url': ['https://en.wikipedia.org/wiki/Black-body_radiation',
  'https://www.youtube.com/watch?v=VydPQuLyEns',
  'https://www.google.com/search?q=cross-section+of+earth',
  'https://upload.wikimedia.org/wikipedia/commons/0/00/Blackbody-colours-vertical.svg'],
 'document': '',
 'q_id': '3onkte',
 'selftext': '[deleted]',
 'selftext_urls.url': [],
 'subreddit': 'askscience',
 'title': 'If Earth was cut in half, what colours would the cross section be?',
 'title_urls.url': []}

In [8]:
def preprocess_function(examples):
  return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [9]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Parameter 'function'=<function preprocess_function at 0x7fdaa35d2c20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1070 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1382 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1659 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1067 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
block_size = 128


def group_texts(examples):
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  total_length = (total_length // block_size) * block_size
  result = {
    k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
    for k, t in concatenated_examples.items()
  }
  result["labels"] = result["input_ids"].copy()
  return result

In [11]:
lm_dataset = tokenized_eli5.map(
  group_texts,
  batched=True,
  num_proc=4
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
# 检查其中一个数据
tokenizer.decode(lm_dataset["train"][1]["input_ids"])

"imeter claims to be zero current).  A typical multimeter operating in voltage mode uses a high impedance circuit, but there still must be some minimum current in order for a signal to be read.  Without a salt bridge to provide charge balance for the electrochemical reaction taking place, the reaction won't proceed far enough for that minimum current to be produced.The precision is way too high.  I've heard of sleep cycles like this, but there's no way everyone has sleep cycles of the exact same length.They've been taxidermied before storage.\n\nThese photos are of the bird skins preserved and stretched over molds"

In [13]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

In [14]:
training_args = TrainingArguments(
  "eli5-finetuned",
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  weight_decay=0.01
)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=lm_dataset["train"],
  eval_dataset=lm_dataset["test"]
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9123
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3423


Epoch,Training Loss,Validation Loss
1,3.871800,3.753867
2,3.775500,3.736330
3,3.724900,3.733263


Saving model checkpoint to eli5-finetuned/checkpoint-500
Configuration saved in eli5-finetuned/checkpoint-500/config.json
Model weights saved in eli5-finetuned/checkpoint-500/pytorch_model.bin
Saving model checkpoint to eli5-finetuned/checkpoint-1000
Configuration saved in eli5-finetuned/checkpoint-1000/config.json
Model weights saved in eli5-finetuned/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2152
  Batch size = 8
Saving model checkpoint to eli5-finetuned/checkpoint-1500
Configuration saved in eli5-finetuned/checkpoint-1500/config.json
Model weights saved in eli5-finetuned/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to eli5-finetuned/checkpoint-2000
Configuration saved in eli5-finetuned/checkpoint-2000/config.json
Model weights saved in eli5-finetuned/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2152
  Batch size = 8
Saving model checkpoint to eli5-finetuned/checkpoint-2500
Configuration saved i

TrainOutput(global_step=3423, training_loss=3.8012129359867988, metrics={'train_runtime': 750.0717, 'train_samples_per_second': 36.489, 'train_steps_per_second': 4.564, 'total_flos': 893928845869056.0, 'train_loss': 3.8012129359867988, 'epoch': 3.0})

In [16]:
# 使用训练后的模型
from transformers import pipeline

model = model.to("cpu")
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
generator("Three Rings for the Elven-kings under the sky, Seven for the Dwarf-lords in their halls of stone")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Three Rings for the Elven-kings under the sky, Seven for the Dwarf-lords in their halls of stone and Six for the Highs in their castles. A dragon could go so far as to take over the Iron City and control the entire'}]